# imports

In [ ]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *

from TCR_feature_generation_functions import *

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'


In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

# run feature extraction functions on pnp530 samples:

### first, calculate nTemplates for cardioSamples


In [ ]:
data_folder='TCR_real_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
nTemplates=pd.DataFrame() #generate empty dataframe

#there is a mistake in the column names in the batch released on 25/2/18:
#get right column names:
f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis_corrected/BD438.tsv'
BD438=pd.read_table(f)
right_column_names=BD438.columns.values
newColumnList=right_column_names[:-3]

for n, sample_name in enumerate(filenames):
    print n,sample_name
    sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected/%s.tsv" %(data_folder,sample_name))
    if newColumnList is not None: #make sure column names are correct:
        sample_df=sample_df.iloc[:,:44]
        sample_df.columns=newColumnList
    sample_df=sample_df.rename(columns={'count (templates/reads)':'count (templates)'})

    sample_df['prod_stat']=np.where(sample_df['sequenceStatus'] == 'In',1,0)
    interesting_columns=['nucleotide','aminoAcid','count (templates)','frequencyCount (%)','cdr3Length',
                         'vDeletion', 'n1Insertion', 'd5Deletion', 'd3Deletion',
                         'n2Insertion','jDeletion','prod_stat']    

    sample_df=sample_df[interesting_columns]
    prod_df=sample_df[sample_df['prod_stat']==1]
    nonprod_df=sample_df[sample_df['prod_stat']==0]
    
    n_templates_total_nt=sample_df['count (templates)'].sum()
    n_templates_prod_nt=prod_df['count (templates)'].sum()
    n_templates_nonprod_nt=nonprod_df['count (templates)'].sum()
    n_templates_total_aa=sample_df.groupby('aminoAcid').sum()['count (templates)'].sum()
    n_templates_prod_aa=prod_df.groupby('aminoAcid').sum()['count (templates)'].sum()
    n_templates_nonprod_aa=nonprod_df.groupby('aminoAcid').sum()['count (templates)'].sum()
        
    
    nTemplates.loc[n,'Sample']=sample_name
    
    nTemplates.loc[n,'n_templates_total_nt']=n_templates_total_nt
    nTemplates.loc[n,'n_templates_prod_nt']=n_templates_prod_nt
    nTemplates.loc[n,'n_templates_nonprod_nt']=n_templates_nonprod_nt
    
    nTemplates.loc[n,'n_temp_total_aa']=n_templates_total_aa
    nTemplates.loc[n,'n_templates_prod_aa']=n_templates_prod_aa
    nTemplates.loc[n,'n_templates_nonprod_aa']=n_templates_nonprod_aa
    
    
    

In [ ]:
#save n templates dataframe
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/nTemplatesSummary_%s_%s' %(len(filenames), cdate)
nTemplates.to_pickle(file1)

### now run feature calculations:

In [ ]:
data_folder='TCR_real_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

# #there is a mistake in the column names in the batch released on 25/2/18:
# #get right column names:
# f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis_corrected/BD438.tsv'
# BD438=pd.read_table(f)
# right_column_names=BD438.columns.values
# newColumnList=right_column_names[:-3]

for n,sample_name in enumerate(filenames): 
    if n<3:
        print  n,sample_name
        gen_descriptive_stats(sample_name,data_folder,newColumnList=None)
        gen_geneUsageCount(sample_name,data_folder,newColumnList=None)

## plot sample percentiles according to number of templates:

the function plot_nTemplates_percentile was copied to TCR_feature_generation_functions.py

In [ ]:
def plot_nTemplates_percentile(nTemplates, datasetName,data_folder):
    templates=list(nTemplates['n_templates_total_nt'])
    ymax=roundup(np.max(templates),10000)
    fig,ax=plt.subplots(figsize=(12,9))
    perc=[0,1,2,5,7.5,10,15,20,30,40,50,60,70,80,90,95,99,100]
    temps=[]
    for n in perc:
        print n,np.percentile(templates,n)
        temps.append(np.percentile(templates,n))
    ax.plot(perc,temps)
    percToShow=[1,5,15]
    tempsToShow=[]
    for p in percToShow:
        ind=perc.index(p)
        t=temps[ind]
        ax.scatter(p,t,color='r',s=20)
        ax.annotate('%sperc-%s templates' %(p,t), xy=(p,t),
                             xytext=(p * 1.05, t * 1.05), fontsize=8,color='r')
    ax.set_xlabel('percent of samples',fontsize=20)
    ax.set_ylabel('number of templates',fontsize=20)
    ax.set_title('Cumulative distribution of #templates per sample - %s' %datasetName,fontsize=26)
    ax.set_xlim(0,100)
    ax.set_ylim(0,ymax)
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/realAnalysis/CumDistNtemplates_%s' %(data_folder,datasetName)
    fig.savefig(file1,dpi=200)
    
    return fig
    

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/nTemplatesSummary_515_08042018' 
nTemplates=pd.read_pickle(file1)

nTemplates=nTemplates
datasetName='PNP515'
data_folder='TCR_real_data'

fig=plot_nTemplates_percentile(nTemplates, datasetName,data_folder)
plt.show()

In [ ]:
plt.hist(templates,bins=100)
plt.xticks(range(0,100000,5000),rotation=90)
plt.show()

# run feature extraction functions on Cardio126 samples:

### first, calculate nTemplates for cardioSamples


In [ ]:
data_folder='TCR_real_data/CardioSamples'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
nTemplates=pd.DataFrame() #generate empty dataframe

#there is a mistake in the column names in the batch released on 25/2/18:
#get right column names:
f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis/BD438.tsv'
BD438=pd.read_table(f)
right_column_names=BD438.columns.values
newColumnList=right_column_names[:-3]

for n, sample_name in enumerate(filenames):
    print n,sample_name
    sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/%s.tsv" %(data_folder,sample_name))
    if newColumnList is not None: #make sure column names are correct:
        sample_df=sample_df.iloc[:,:44]
        sample_df.columns=newColumnList
    sample_df=sample_df.rename(columns={'count (templates/reads)':'count (templates)'})

    sample_df['prod_stat']=np.where(sample_df['sequenceStatus'] == 'In',1,0)
    interesting_columns=['nucleotide','aminoAcid','count (templates)','frequencyCount (%)','cdr3Length',
                         'vDeletion', 'n1Insertion', 'd5Deletion', 'd3Deletion',
                         'n2Insertion','jDeletion','prod_stat']    

    sample_df=sample_df[interesting_columns]
    prod_df=sample_df[sample_df['prod_stat']==1]
    nonprod_df=sample_df[sample_df['prod_stat']==0]
    
    n_templates_total_nt=sample_df['count (templates)'].sum()
    n_templates_prod_nt=prod_df['count (templates)'].sum()
    n_templates_nonprod_nt=nonprod_df['count (templates)'].sum()
    n_templates_total_aa=sample_df.groupby('aminoAcid').sum()['count (templates)'].sum()
    n_templates_prod_aa=prod_df.groupby('aminoAcid').sum()['count (templates)'].sum()
    n_templates_nonprod_aa=nonprod_df.groupby('aminoAcid').sum()['count (templates)'].sum()
        
    
    nTemplates.loc[n,'Sample']=sample_name
    
    nTemplates.loc[n,'n_templates_total_nt']=n_templates_total_nt
    nTemplates.loc[n,'n_templates_prod_nt']=n_templates_prod_nt
    nTemplates.loc[n,'n_templates_nonprod_nt']=n_templates_nonprod_nt
    
    nTemplates.loc[n,'n_temp_total_aa']=n_templates_total_aa
    nTemplates.loc[n,'n_templates_prod_aa']=n_templates_prod_aa
    nTemplates.loc[n,'n_templates_nonprod_aa']=n_templates_nonprod_aa
    
    
    

In [ ]:
#save n templates dataframe
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/nTemplatesSummary_%s_%s' %(len(filenames), cdate)
nTemplates.to_pickle(file1)

In [ ]:
nTemplates.describe()

conclusion: using 2000 as minimal number of nucleotides and 200 as minimal number of aa is ok. 

### generate nTemplates cumulative distribution for Cardio91:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/nTemplatesSummary_91_08042018' 
nTemplates=pd.read_pickle(file1)

nTemplates=nTemplates
datasetName='Cardio91'
data_folder='TCR_real_data/CardioSamples'

fig=plot_nTemplates_percentile(nTemplates, datasetName,data_folder)
plt.show()

### now run feature calculations:

In [ ]:
data_folder='TCR_real_data/CardioSamples'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

# #there is a mistake in the column names in the batch released on 25/2/18:
# #get right column names:
# f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis/BD438.tsv'
# BD438=pd.read_table(f)
# right_column_names=BD438.columns.values
# newColumnList=right_column_names[:-3]

for n,sample_name in enumerate(filenames): 
#     if n>3:
        print  n,sample_name
        gen_descriptive_stats(sample_name,data_folder,newColumnList=None)
        gen_geneUsageCount(sample_name,data_folder,newColumnList=None)

# run feature extraction functions on Ravidsamples:

### first, calculate nTemplates for cardioSamples


In [ ]:
data_folder='TCR_real_data/RavidSamples'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
nTemplates=pd.DataFrame() #generate empty dataframe

# #there is a mistake in the column names in the batch released on 25/2/18:
# #get right column names:
# f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis_corrected/BD438.tsv'
# BD438=pd.read_table(f)
# right_column_names=BD438.columns.values
# newColumnList=right_column_names[:-3]

for n, sample_name in enumerate(filenames):
    print n,sample_name
    sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected/%s.tsv" %(data_folder,sample_name))
#     if newColumnList is not None: #make sure column names are correct:
#         sample_df=sample_df.iloc[:,:44]
#         sample_df.columns=newColumnList
    sample_df=sample_df.rename(columns={'count (templates/reads)':'count (templates)'})

    sample_df['prod_stat']=np.where(sample_df['sequenceStatus'] == 'In',1,0)
    interesting_columns=['nucleotide','aminoAcid','count (templates)','frequencyCount (%)','cdr3Length',
                         'vDeletion', 'n1Insertion', 'd5Deletion', 'd3Deletion',
                         'n2Insertion','jDeletion','prod_stat']    

    sample_df=sample_df[interesting_columns]
    prod_df=sample_df[sample_df['prod_stat']==1]
    nonprod_df=sample_df[sample_df['prod_stat']==0]
    
    n_templates_total_nt=sample_df['count (templates)'].sum()
    n_templates_prod_nt=prod_df['count (templates)'].sum()
    n_templates_nonprod_nt=nonprod_df['count (templates)'].sum()
    n_templates_total_aa=sample_df.groupby('aminoAcid').sum()['count (templates)'].sum()
    n_templates_prod_aa=prod_df.groupby('aminoAcid').sum()['count (templates)'].sum()
    n_templates_nonprod_aa=nonprod_df.groupby('aminoAcid').sum()['count (templates)'].sum()
        
    
    nTemplates.loc[n,'Sample']=sample_name
    
    nTemplates.loc[n,'n_templates_total_nt']=n_templates_total_nt
    nTemplates.loc[n,'n_templates_prod_nt']=n_templates_prod_nt
    nTemplates.loc[n,'n_templates_nonprod_nt']=n_templates_nonprod_nt
    
    nTemplates.loc[n,'n_temp_total_aa']=n_templates_total_aa
    nTemplates.loc[n,'n_templates_prod_aa']=n_templates_prod_aa
    nTemplates.loc[n,'n_templates_nonprod_aa']=n_templates_nonprod_aa
    
    
    

In [ ]:
#save n templates dataframe
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/RavidSamples/nTemplatesSummary_%s_%s' %(len(filenames), cdate)
nTemplates.to_pickle(file1)

In [ ]:
nTemplates.describe()

conclusion: using 2000 as minimal number of nucleotides and 200 as minimal number of aa is ok. 

### generate nTemplates cumulative distribution for Cardio91:

In [ ]:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/RavidSamples/nTemplatesSummary_4_13062018' 
nTemplates=pd.read_pickle(file1)

nTemplates=nTemplates
datasetName='RavidSamples'
data_folder='TCR_real_data/RavidSamples'

fig=plot_nTemplates_percentile(nTemplates, datasetName,data_folder)
plt.show()

### now run feature calculations:

In [ ]:
data_folder='TCR_real_data/RavidSamples'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis_corrected' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

# #there is a mistake in the column names in the batch released on 25/2/18:
# #get right column names:
# f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis/BD438.tsv'
# BD438=pd.read_table(f)
# right_column_names=BD438.columns.values
# newColumnList=right_column_names[:-3]

for n,sample_name in enumerate(filenames): 
#     if n>3:
        print  n,sample_name
        gen_descriptive_stats(sample_name,data_folder,newColumnList=None)
        gen_geneUsageCount(sample_name,data_folder,newColumnList=None)

# run feature extraction functions on PNP_ss18000 samples:

In [ ]:
nTemplates=18000
data_folder='TCR_real_data/SubSampled%s data' %nTemplates ## change here for the relevant folder


dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
filenames=[f.strip('.xlsx') for f in filenames]
print len(filenames)

# #there is a mistake in the column names in the batch released on 25/2/18:
# #get right column names:
# f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/BD438_%s.xlsx' %(data_folder, nTemplates)
# BD438=pd.read_excel(f)
# right_column_names=BD438.columns.values
newColumnList=None

for n,sample_name in enumerate(filenames): 
    if n>91: 
        print  n,sample_name
        if 'nSampled' not in sample_name:
            gen_descriptive_stats(sample_name,data_folder,newColumnList)
            gen_geneUsageCount(sample_name,data_folder,newColumnList)

# run feature extraction functions on PNP515_ss12500 samples:

In [ ]:
nTemplates=12500
data_folder='TCR_real_data/SubSampled%s data' %nTemplates ## change here for the relevant folder


dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
filenames=[f.strip('.xlsx') for f in filenames]
print len(filenames)

# #there is a mistake in the column names in the batch released on 25/2/18:
# #get right column names:
# f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/BD438_%s.xlsx' %(data_folder, nTemplates)
# BD438=pd.read_excel(f)
# right_column_names=BD438.columns.values
newColumnList=None

for n,sample_name in enumerate(filenames): 
#     if n<92: 
        print  n,sample_name
        if 'nSampled' not in sample_name:
            gen_descriptive_stats(sample_name,data_folder,newColumnList)
            gen_geneUsageCount(sample_name,data_folder,newColumnList)

# run feature extraction functions on PNP515_ss9000 samples:

In [ ]:
nTemplates=9000
data_folder='TCR_real_data/SubSampled%s data' %nTemplates ## change here for the relevant folder


dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
filenames=[f.strip('.xlsx') for f in filenames]
print len(filenames)

# #there is a mistake in the column names in the batch released on 25/2/18:
# #get right column names:
# f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/BD438_%s.xlsx' %(data_folder, nTemplates)
# BD438=pd.read_excel(f)
# right_column_names=BD438.columns.values
newColumnList=None

for n,sample_name in enumerate(filenames): 
#     if n<92: 
        print  n,sample_name
        if 'nSampled' not in sample_name:
            gen_descriptive_stats(sample_name,data_folder,newColumnList)
            gen_geneUsageCount(sample_name,data_folder,newColumnList)

# run feature extraction functions on REPEATED PNP515_ss9000 samples:

In [ ]:
nTemplates=9000
data_folder='TCR_real_data/SubSampled%sdataRepeated' %nTemplates ## change here for the relevant folder


dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
filenames=[f.strip('.xlsx') for f in filenames]
print len(filenames)

# #there is a mistake in the column names in the batch released on 25/2/18:
# #get right column names:
# f='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/BD438_%s.xlsx' %(data_folder, nTemplates)
# BD438=pd.read_excel(f)
# right_column_names=BD438.columns.values
newColumnList=None

for n,sample_name in enumerate(filenames): 
#     if n>91: 
        print  n,sample_name
        if 'nSampled' not in sample_name:
            gen_descriptive_stats(sample_name,data_folder,newColumnList)
            gen_geneUsageCount(sample_name,data_folder,newColumnList)

# develop feature extraction statistics:

all functions below were copied to 'TCR_feature_generation_functions.py' -use from there and
update there!

## define functions to calculate statistics

### non-gene usage functions:

In [ ]:
# this function gets as input the tuple (df type, df data)
#df type can be 'Total', 'Prod' or 'nonProd', and the df data is the dataframe itself. 
def gen_LengthFeaturesAndMore(df):
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/LengthFeaturesAndMore/%s' %(data_folder,df[0],sample_name) 
    dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/LengthFeaturesAndMore' %(data_folder,df[0])
    if not isdir(dfs_folder):
        mkdir(dfs_folder) 
    files=[f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]

    
    if sample_name not in files:
        print('calculating length features...')
        DF=df[1] #DF is only the data
        DF=DF.drop(['nucleotide','aminoAcid'],axis=1) #drop those columns as they interfere with the mean,max,std calculations
        LengthFeaturesAndMore=pd.DataFrame() #generate empty dataframe
        LengthFeaturesAndMore.loc[0,'Sample']=sample_name 
        LengthFeaturesAndMore.loc[0,'df type']=df[0]
        for column in DF.columns.values: #loop over each column and calculate its mean, std and max, store information in the dataframe
            LengthFeaturesAndMore.loc[0,'%s_mean' %column]=DF[column].mean()
            LengthFeaturesAndMore.loc[0,'%s_std' %column]=DF[column].std()
            LengthFeaturesAndMore.loc[0,'%s_max' %column]=DF[column].max()
    #     if df[0]=='Total':
    #         LengthFeaturesAndMore.loc[0,'perc_prod']=DF['prod_stat'].mean()
    #     else:
    #         LengthFeaturesAndMore.loc[0,'perc_prod']=np.nan

        #save dataframe for each df type in each sample that contains length features:
        LengthFeaturesAndMore.to_pickle(file1)
    else:
        print('found length features for this sample...')

In [ ]:

# calculate the mean %gc in all NT sequences in a specific df:
#this function is used within gen_generalFeatures
def gc_content(df):
    seqs=list(df['nucleotide'])
    gc_values = [GC(seq) for seq in seqs]
    mean_gc=np.mean(gc_values)
    return mean_gc


# this function gets as input the tuple (df type, df data)
#df type can be 'Total', 'Prod' or 'nonProd', and the df data is the dataframe itself. 

def gen_generalFeatures(df):
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/GeneralFeatures/%s' %(data_folder,df[0],sample_name)   
    dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/GeneralFeatures' %(data_folder,df[0])
    if not isdir(dfs_folder):
            mkdir(dfs_folder)    
    files=[f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
    
    if sample_name not in files:
        print('calculating general features...')

        generalFeatures=pd.DataFrame() #generate empty dataframe
        generalFeatures.loc[0,'Sample']=sample_name
        generalFeatures.loc[0,'df type']=df[0]
        #calculate general features:
        generalFeatures.loc[0,'NT count']=len(df[1])
        n_aa_in_total=len(df[1].groupby('aminoAcid'))
        generalFeatures.loc[0,'AA count']=n_aa_in_total
        nt_per_aa=df[1].groupby(['aminoAcid'])[['nucleotide']].count() #for each aa sequence, count how many nt sequence generate the same one
        max_nt_per_aa=max(nt_per_aa['nucleotide'])
        mean_nt_per_aa=round(np.mean(nt_per_aa['nucleotide']),3)
        generalFeatures.loc[0,'max_nt_per_aa']=max_nt_per_aa
        generalFeatures.loc[0,'mean_nt_per_aa']=mean_nt_per_aa
        generalFeatures.loc[0,'gc_content']=gc_content(df[1])


        #save dataframe for each df type in each sample that contains general  features:
        generalFeatures.to_pickle(file1)
    
    else:
        print('found General Features for this sample...')


In [ ]:
#this function counts the number of unique sequences per defined number of templates, using random sampling without replacement:
def norm_uniqe_nt_sequences(df): 
    repeats=10
    samp_size=2000 # this number was selected based on the PNP cohort data, can be changed but need to change the column name in the df!
    reads=list(df['count (templates)'])
    df=df.set_index('nucleotide')
    seqs=[str(i) for i in list(df.index)]
    seq_popped=[]
    for i in range(0,len(seqs)):
        for j in range(0,reads[i]):
            seq_popped.append(seqs[i])        
    seq_n_list=[]
    n_templates_NT=len(seq_popped)
    if len(seq_popped)>samp_size:
        for t in range(repeats): #as the calculation has a random component, repeat this calculation several times and average
            rand_seq=np.random.choice(seq_popped, samp_size, replace=False)
            seq_n=len(set(rand_seq))
            seq_n_list.append(seq_n)
            mean_seq_n=np.mean(seq_n_list)
    else:
        mean_seq_n=np.NaN
        
    return mean_seq_n 



#this function counts the number of unique sequences per defined number of templates, using random sampling without replacement:
def norm_uniqe_aa_sequences(df): 
    repeats=10
    samp_size=200 # this number was selected based on the PNP cohort data, can be changed but need to change the column name in the df!
    reads=list(df['count (templates)'])
    list_aa=list(df['aminoAcid'])  
    seq_aa_popped=[]
    for i in range(0,len(list_aa)):
        for j in range(0,reads[i]):
            seq_aa_popped.append(list_aa[i])
    seq_aa_popped_new=[i for i in seq_aa_popped if isinstance(i, str)]
    seq_n_list=[]
    n_templates_aa=len(seq_aa_popped_new)
    if len(seq_aa_popped_new)>samp_size:
        for t in range(repeats): #as the calculation has a random component, repeat this calculation several times and average
            rand_seq=np.random.choice(seq_aa_popped_new, samp_size, replace=False)
            seq_n=len(set(rand_seq))
            seq_n_list.append(seq_n)
        mean_seq_n=np.mean(seq_n_list)
    else:
        mean_seq_n=np.NaN
    return mean_seq_n         



def gen_normSeqNums(df):
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/normSeqNums/%s' %(data_folder,df[0],sample_name)   
    dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/normSeqNums'  %(data_folder, df[0])
    if not isdir(dfs_folder):
            mkdir(dfs_folder)  
    files=[f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
    
    if sample_name not in files:
        print('calculating normSeqNums features...')

        normSeqNums=pd.DataFrame() #generate empty dataframe
        normSeqNums.loc[0,'Sample']=sample_name
        normSeqNums.loc[0,'df type']=df[0]

        DF=df[1]

        #calculate general features:
        normSeqNums.loc[0,'normSeqNums_per2000_NT']=norm_uniqe_nt_sequences(DF)
        normSeqNums.loc[0,'normSeqNums_per200_AA']=norm_uniqe_aa_sequences(DF)

        #save dataframe for each df type in each sample that contains general  features:

        normSeqNums.to_pickle(file1)
    else:
        print('found normSeqNums for this sample...')
    
    
    
    

In [ ]:
# this function gets as input the tuple (df type, df data)
#df type can be 'Total', 'Prod' or 'nonProd', and the df data is the dataframe itself. 

def gen_clonalityFeatures(df):
    
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/ClonalityFeatures/%s' %(data_folder,df[0],sample_name)
    dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/ClonalityFeatures' %(data_folder,df[0])
    if not isdir(dfs_folder):
            mkdir(dfs_folder)   
    files=[f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
    
    if sample_name not in files:
        print('calculating clonality features...')

        DF=df[1] #DF is only the data
        DF_aa=pd.DataFrame(DF.groupby('aminoAcid').sum()['frequencyCount (%)'])
        DF['clonality_nt']=DF['frequencyCount (%)']/DF['frequencyCount (%)'].sum()
        DF_aa['clonality_aa']=DF_aa['frequencyCount (%)']/DF_aa['frequencyCount (%)'].sum()
        clonalityList_nt=list(DF['clonality_nt'].sort_values(ascending=False))
        clonalityList_aa=list(DF_aa['clonality_aa'].sort_values(ascending=False))

        if len(clonalityList_nt)<100:
            top100clonal_nt=np.nan
        else:
            top100clonal_nt=np.sum(clonalityList_nt[:100])


        if len(clonalityList_aa)<100:
            top100clonal_aa=np.nan
        else:
            top100clonal_aa=np.sum(clonalityList_nt[:100])

        topClonal_nt=np.max(clonalityList_nt)
        meanClonal_nt=np.mean(clonalityList_nt)
        medianClonal_nt=np.median(clonalityList_nt)
        stdClonal_nt=np.std(clonalityList_nt)
        percentile1_nt=np.percentile(clonalityList_nt,1)
        percentile999_nt=np.percentile(clonalityList_nt,99.9)

        topClonal_aa=np.max(clonalityList_aa)
        meanClonal_aa=np.mean(clonalityList_aa)
        medianClonal_aa=np.median(clonalityList_aa)
        stdClonal_aa=np.std(clonalityList_aa)
        percentile1_aa=np.percentile(clonalityList_aa,1)
        percentile999_aa=np.percentile(clonalityList_aa,99.9)

        clonFeatures=pd.DataFrame() #generate empty dataframe
        clonFeatures.loc[0,'Sample']=sample_name
        clonFeatures.loc[0,'df type']=df[0]
        clonFeatures.loc[0,'topClonal_nt']=topClonal_nt
        clonFeatures.loc[0,'meanClonal_nt']=meanClonal_nt
        clonFeatures.loc[0,'medianClonal_nt']=medianClonal_nt
        clonFeatures.loc[0,'stdClonal_nt']=stdClonal_nt
        clonFeatures.loc[0,'percentile1_nt']=percentile1_nt
        clonFeatures.loc[0,'percentile999_nt']=percentile999_nt
        clonFeatures.loc[0,'topClonal_aa']=topClonal_aa
        clonFeatures.loc[0,'meanClonal_aa']=meanClonal_aa
        clonFeatures.loc[0,'medianClonal_aa']=medianClonal_aa
        clonFeatures.loc[0,'stdClonal_aa']=stdClonal_aa
        clonFeatures.loc[0,'percentile1_aa']=percentile1_aa
        clonFeatures.loc[0,'percentile999_aa']=percentile999_aa

        #save dataframe for each df type in each sample that contains general  features:
        clonFeatures.to_pickle(file1)
        
    else:
        print('found clonality features for this sample...')


In [ ]:
# this function gets as input the tuple (df type, df data)
#df type can be 'Total', 'Prod' or 'nonProd', and the df data is the dataframe itself. 

def gen_diversityFeatures(df):
    
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/DiversityFeatures/%s' %(data_folder,df[0],sample_name)
    dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/DiversityFeatures' %(data_folder,df[0])
    if not isdir(dfs_folder):
            mkdir(dfs_folder)
    files=[f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
    
    if sample_name not in files:
        print('calculating diversity features...')

        DF=df[1] #DF is only the data
        DF_aa=pd.DataFrame(DF.groupby('aminoAcid').sum()['count (templates)'])

        shannon_nt=shannon(DF['count (templates)'], base=2)
        simpson_nt=simpson(DF['count (templates)'])
        berger_nt=berger_parker_d(DF['count (templates)'])
        shannon_aa=shannon(DF_aa['count (templates)'], base=2)
        simpson_aa=simpson(DF_aa['count (templates)'])
        berger_aa=berger_parker_d(DF_aa['count (templates)'])

        diversityFeatures=pd.DataFrame() #generate empty dataframe
        diversityFeatures.loc[0,'Sample']=sample_name
        diversityFeatures.loc[0,'df type']=df[0]
        diversityFeatures.loc[0,'shannon_nt']=shannon_nt
        diversityFeatures.loc[0,'simpson_nt']=simpson_nt
        diversityFeatures.loc[0,'berger_nt']=berger_nt
        diversityFeatures.loc[0,'shannon_aa']=shannon_aa
        diversityFeatures.loc[0,'simpson_aa']=simpson_aa
        diversityFeatures.loc[0,'berger_aa']=berger_aa

        #save dataframe for each df type in each sample that contains general  features:
        diversityFeatures.to_pickle(file1)
    else:
        print('found diversityFeatures for this sample...')



### gene usage functions:

In [ ]:
## this function counts independent gene usage **without unresolved!!**:


def count_geneUsage(df):
    print('counting gene usage features...')
    
    param_list=['vGeneName','vFamilyName','dFamilyName','jGeneName']
    DF=df[1]
    
    for param in param_list:
        file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/%s/%s' %(data_folder, df[0],param,sample_name)
        dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/%s/' %(data_folder,df[0],param)
        if not isdir(dfs_folder):
            mkdir(dfs_folder)
        files=[f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
    
        if sample_name not in files:
            print('calculating gene usage for param %s' %param)
            GeneUsage=pd.DataFrame(DF[param].value_counts(normalize=True))
            GeneUsage=GeneUsage.rename(columns={'%s' %param: '%s' %sample_name})
            GeneUsage=GeneUsage.T
            GeneUsage.loc[sample_name,'df type']=df[0]
            GeneUsage.to_pickle(file1)
        else:
            print('found gene usage count for %s' %param)

            
## this function counts dependent (combined) gene usage **without unresolved!!**:
def count_dependent_geneUsage(df):
    print('counting dependent gene usage features...')
    
    dep_param_list=['V-J family combination','D-J gene combination']
    DF=df[1]
    DF['V-J family combination']=DF['vFamilyName']+'_'+DF['jFamilyName']
    DF['D-J gene combination']=DF['dFamilyName']+'_'+DF['jGeneName']
    
    for dep_param in dep_param_list:
        file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/%s/%s' %(data_folder,df[0],dep_param,sample_name)        
        dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/descriptiveStatsSamplesForAnalysis/%s/%s/' %(data_folder, df[0],dep_param)
        
        if not isdir(dfs_folder):
            mkdir(dfs_folder) 
        files=[f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
    
        if sample_name not in files:
            print('calculating dep gene usage for dep param %s' %dep_param)

            depGeneUsage=pd.DataFrame(DF[dep_param].value_counts(normalize=True))
            depGeneUsage=depGeneUsage.rename(columns={'%s' %dep_param: '%s' %sample_name})
            depGeneUsage=depGeneUsage.T
            depGeneUsage.loc[sample_name,'df type']=df[0]
            depGeneUsage.to_pickle(file1)
        else:
            print('found dep gene usage count for %s' %dep_param)        

    

### playground:

In [ ]:
#################playground#############
param_list=['vGeneName','vFamilyName','dGeneName','jGeneName']
sample_name='BD2'
data_folder='TCR_real_data'

sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/%s.tsv" %(data_folder,sample_name))
sample_df=sample_df.rename(columns={'count (templates/reads)':'count (templates)'})
sample_df['prod_stat']=np.where(sample_df['sequenceStatus'] == 'In',1,0)
    
interesting_columns=['vGeneName','vFamilyName','dFamilyName','jGeneName','jFamilyName','prod_stat']       

sample_df=sample_df[interesting_columns]
    
for column in sample_df.columns.values:
    if sample_df[column].dtype!='int64':
        sample_df=sample_df[sample_df[column].str.contains('unresolved')==False]
        sample_df[column]=sample_df[column].str.replace('TCRB','')
sample_df['V-J family combination']=sample_df['vFamilyName']+'_'+sample_df['jFamilyName']
sample_df['D-J gene combination']=sample_df['dFamilyName']+'_'+sample_df['jGeneName']
sample_df

# sample_df=sample_df[['jGeneName','dGeneName']]
# sample_df=sample_df[samples_df['jGeneName'] != 'unresolved']
# sample_df=sample_df[samples_df['dGeneName'] != 'unresolved']

# VJusage=pd.crosstab(index=sample_df['jGeneName'], columns=sample_df['dGeneName'], normalize='index')

# VJusage
    

        




In [ ]:
data_folder=r'TCR_demo_data'
sample_name='HIP00110'
gen_descriptive_stats(sample_name)


### calculate n templates in each sample and each df type:

In [ ]:
nTemplates=pd.DataFrame() #generate empty dataframe



for n, sample_name in enumerate(filenames):
    print n,sample_name
    sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamplesForAnalysis/%s.tsv" %sample_name)
    sample_df=sample_df.rename(columns={'count (templates/reads)':'count (templates)'})

    sample_df['prod_stat']=np.where(sample_df['sequenceStatus'] == 'In',1,0)
    interesting_columns=['nucleotide','aminoAcid','count (templates)','frequencyCount (%)','cdr3Length',
                         'vDeletion', 'n1Insertion', 'd5Deletion', 'd3Deletion',
                         'n2Insertion','jDeletion','prod_stat']    

    sample_df=sample_df[interesting_columns]
    prod_df=sample_df[sample_df['prod_stat']==1]
    nonprod_df=sample_df[sample_df['prod_stat']==0]
    
    n_templates_total_nt=sample_df['count (templates)'].sum()
    n_templates_prod_nt=prod_df['count (templates)'].sum()
    n_templates_nonprod_nt=nonprod_df['count (templates)'].sum()
    n_templates_total_aa=sample_df.groupby('aminoAcid').sum()['count (templates)'].sum()
    n_templates_prod_aa=prod_df.groupby('aminoAcid').sum()['count (templates)'].sum()
    n_templates_nonprod_aa=nonprod_df.groupby('aminoAcid').sum()['count (templates)'].sum()
        
    
    nTemplates.loc[n,'Sample']=sample_name
    
    nTemplates.loc[n,'n_templates_total_nt']=n_templates_total_nt
    nTemplates.loc[n,'n_templates_prod_nt']=n_templates_prod_nt
    nTemplates.loc[n,'n_templates_nonprod_nt']=n_templates_nonprod_nt
    
    nTemplates.loc[n,'n_temp_total_aa']=n_templates_total_aa
    nTemplates.loc[n,'n_templates_prod_aa']=n_templates_prod_aa
    nTemplates.loc[n,'n_templates_nonprod_aa']=n_templates_nonprod_aa
    
    
    

In [ ]:
#save n templates dataframe
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/nTemplatesSummary_%s_%s' %(len(filenames), cdate)
nTemplates.to_pickle(file1)

In [ ]:
nTemplates.describe()

## define highest function to call all feature calculating functions:

In [ ]:
def gen_descriptive_stats(sample_name):
    # (1) read sample data, add indications for productive, get only interesting
    # columns and generate dfs for total, only productive, only non productive

    sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/%s.tsv" %(data_folder,sample_name))
    sample_df=sample_df.rename(columns={'count (templates/reads)':'count (templates)','count (reads)':'count (templates)'})

    sample_df['prod_stat']=np.where(sample_df['sequenceStatus'] == 'In',1,0)
    interesting_columns=['nucleotide','aminoAcid','count (templates)','frequencyCount (%)','cdr3Length',
                         'vDeletion', 'n1Insertion', 'd5Deletion', 'd3Deletion',
                         'n2Insertion','jDeletion','prod_stat']    

    sample_df=sample_df[interesting_columns]
    prod_df=sample_df[sample_df['prod_stat']==1]
    nonprod_df=sample_df[sample_df['prod_stat']==0]
    
    df_dict=[('Total',sample_df),
     ('Prod',prod_df),
     ('nonProd',nonprod_df)]
 
 # (2) call selected functions to calculate statistics for each df
    for df in df_dict:
        print df[0]
        gen_LengthFeaturesAndMore(df)
        gen_generalFeatures(df)
        gen_normSeqNums(df)
        gen_clonalityFeatures(df)
        gen_diversityFeatures(df)

    
    
    
      


In [ ]:
# geneUsage count has its own function as the column needed are different

def gen_geneUsageCount(sample_name):
    # (1) read sample data, add indications for productive, get only interesting
    # columns and generate dfs for total, only productive, only non productive

    sample_df=pd.read_table("/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis/%s.tsv" %(data_folder, sample_name))
    sample_df=sample_df.rename(columns={'count (templates/reads)':'count (templates)','count (reads)':'count (templates)'})
    sample_df['prod_stat']=np.where(sample_df['sequenceStatus'] == 'In',1,0)
    
    interesting_columns=['vGeneName','vFamilyName','dFamilyName','jGeneName','jFamilyName','prod_stat']       

    sample_df=sample_df[interesting_columns]
    for column in sample_df.columns.values: #remove 'unresolved' counts and calculate frequencies without it. remove
                                            # 'TCRB' indication
        if sample_df[column].dtype!='int64':
            sample_df=sample_df[sample_df[column].str.contains('unresolved')==False]
            sample_df[column]=sample_df[column].str.replace('TCRB','')
    
    prod_df=sample_df[sample_df['prod_stat']==1]
    nonprod_df=sample_df[sample_df['prod_stat']==0]
    
    df_dict=[('Total',sample_df),
     ('Prod',prod_df),
     ('nonProd',nonprod_df)]
 
 # (2) call selected functions to calculate statistics for each df
    for df in df_dict:
        count_geneUsage(df)
        count_dependent_geneUsage(df)
    
    
    
      


## get sample names and run feature generation functions on them:

In [ ]:
data_folder='TCR_real_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames=[f.strip('.tsv') for f in filenames]
print len(filenames)

In [ ]:
for n,sample_name in enumerate(filenames):    
        print  n,sample_name
        gen_descriptive_stats(sample_name)
#         gen_geneUsageCount(sample_name)

In [ ]:
#calculating for 4 additional samples (duplicates that were reanalyzed as singles)
singles=['BD107_1rep','BD111_1rep','BD113_1rep','BD128_1rep']
data_folder='TCR_real_data'
for n,sample_name in enumerate(singles):    
        print  n,sample_name
        gen_descriptive_stats(sample_name)
        gen_geneUsageCount(sample_name)

## Samuels samples: get sample names and run feature generation functions on them:

In [ ]:
data_folder='TCR_real_data/SamuelsSamples'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames_samuels = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames_samuels=[f.strip('.tsv') for f in filenames_samuels]
print len(filenames_samuels)
print dfs_folder

In [ ]:
for n,sample_name in enumerate(filenames_samuels):
#     if n<3:    
        print  n,sample_name
        gen_descriptive_stats(sample_name)
        gen_geneUsageCount(sample_name)

## run all analysis on Adaptive samples:

In [ ]:
data_folder='TCR_demo_data'
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/%s/SamplesForAnalysis' %data_folder
filenames_adaptive = [f for f in listdir(dfs_folder) if isfile(join(dfs_folder, f))]
filenames_adaptive=[f.strip('.tsv') for f in filenames_adaptive]
print len(filenames_adaptive)
print dfs_folder

In [ ]:
for n,sample_name in enumerate(filenames_adaptive):
    if n>607:    
        print  n,sample_name
        gen_descriptive_stats(sample_name)
        gen_geneUsageCount(sample_name)

In [ ]:
#run 4 samples that were problematic before, and I re-saved them:
problematic=['HIP01129','HIP14211','HIP01947','HIP14092']

for n,sample_name in enumerate(problematic):    
        print  n,sample_name
        gen_descriptive_stats(sample_name)
        gen_geneUsageCount(sample_name)

#### find which adaptive samples needs to be downloaded:

In [ ]:
#find the full list of adaptive samples according to the QC file I downloaded:

file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/qcReport.2017-11-26_03-34-38.tsv'

AdaptiveSamples=pd.read_table(file1)
AdaptiveSamples.head()

In [ ]:
AdaptiveSamplesList=list(AdaptiveSamples['Sample Name'])
print len(AdaptiveSamplesList)

In [ ]:
# find the list of samples for which I already have the data downloaded:
filenames_adaptive[:5]


In [ ]:
#find which samples needs to be downloaded and save to excel:
missing=[x for x in AdaptiveSamplesList if x not in filenames_adaptive]

In [ ]:
print len(missing)

In [ ]:
missingDF=pd.DataFrame(missing)
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/missingSamples.xlsx'
missingDF.to_excel(file2)

## concatenate dfs of the same type:

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/Prod/GeneralFeatures'
Prod_generalFeatures=concat_summarizing_dfs(dfs_folder)

In [ ]:
Prod_generalFeatures.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/nonProd/LengthFeaturesAndMore'
nonProd_lengthFeatures=concat_summarizing_dfs(dfs_folder)

In [ ]:
nonProd_lengthFeatures.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/Prod/normSeqNums'
prod_normSeqNums=concat_summarizing_dfs(dfs_folder)


In [ ]:
prod_normSeqNums.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/nonProd/normSeqNums'
nonProd_normSeqNums=concat_summarizing_dfs(dfs_folder)


In [ ]:
nonProd_normSeqNums.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/Prod/ClonalityFeatures'
Prod_ClonalityFeatures=concat_summarizing_dfs(dfs_folder)


In [ ]:
Prod_ClonalityFeatures.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/nonProd/ClonalityFeatures'
nonProd_ClonalityFeatures=concat_summarizing_dfs(dfs_folder)


In [ ]:
nonProd_ClonalityFeatures.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/Prod/DiversityFeatures'
Prod_DiversityFeatures=concat_summarizing_dfs(dfs_folder)


In [ ]:
Prod_DiversityFeatures.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/nonProd/DiversityFeatures'
nonProd_DiversityFeatures=concat_summarizing_dfs(dfs_folder)


In [ ]:
nonProd_DiversityFeatures.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/Prod/jGeneName'
Prod_jGeneName=concat_summarizing_dfs(dfs_folder)


In [ ]:
Prod_jGeneName.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/nonProd/jGeneName'
nonProd_jGeneName=concat_summarizing_dfs(dfs_folder)


In [ ]:
nonProd_jGeneName.describe()

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamuelsSamples/descriptiveStatsSamplesForAnalysis/nonProd/dGeneName'
Samuels_nonProd_dGeneName=concat_summarizing_dfs(dfs_folder)


In [ ]:
Samuels_nonProd_dGeneName

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/SamuelsSamples/descriptiveStatsSamplesForAnalysis/nonProd/V-J family combination'
Samuels_nonProd_VJfamilycombination=concat_summarizing_dfs(dfs_folder)


In [ ]:
Samuels_nonProd_VJfamilycombination

In [ ]:
dfs_folder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/descriptiveStatsSamplesForAnalysis/nonProd/dFamilyName'
df=concat_summarizing_dfs(dfs_folder)


In [ ]:
df.head()
# df['unresolved'].count()

In [ ]:
V-J family combination

In [ ]:
Samuels_nonProd_VJfamilycombination.sum(axis=1)

# calculate TCR-microbiome diversity interactions:

## old functions:

In [ ]:
def plot_res_df(result_df, function_list, title, yscale, hspace):
    print 'plotting results...'
    result_df.dropna(inplace=True)
    function_names=[f.__name__ for f in function_list]
    fig=plt.figure(figsize=(6,11))
    plt.suptitle('%s population features' %title, fontsize=16)
    n_plots=len(function_list)
    print n_plots
    for p in range(n_plots):
        print p
        function_name=function_names[p]
        prod_col=2*p
        non_prod_col=2*p+1
        ax= plt.subplot2grid((n_plots,1), (p,0)) 
        plot=plot_population_view(ax, p,  result_df, function_name, yscale, prod_col, non_prod_col)
    plt.subplots_adjust(left=0.14,bottom=0.08, right=0.88, top=0.92, wspace=0.24,hspace=hspace)
    return fig


In [ ]:
def plot_1_res_df(result_df, function_list, title,yscale):
    print 'plotting results...'
    result_df.dropna(inplace=True)
    function_names=[f.__name__ for f in function_list]
    fig=plt.figure(figsize=(7,8))
    plt.suptitle('%s population features' %title, fontsize=16)
    n_plots=len(function_list)
    print n_plots
    for p in range(n_plots):
        print p
        function_name=function_names[p]
        df_col=p
        ax= plt.subplot2grid((n_plots,1), (p,0)) 
        plot=plot_population_view(ax, p,  result_df, function_name, yscale, df_col, non_prod_col=None)
    plt.subplots_adjust(left=0.09,bottom=0.11, right=0.95, top=0.89, wspace=0.24,hspace=0.50)
    return fig


In [ ]:
def plot_population_view(ax,p, result_df,function_name, yscale, prod_col,non_prod_col):
      
    prod_l=list(result_df[result_df.columns[prod_col]])
    if non_prod_col!=None:
        non_prod_l=list(result_df[result_df.columns[non_prod_col]])
        plot=ax.hist((prod_l,non_prod_l), bins=50, color=('blue', 'red'), label=('Productive','Non-Productive'), alpha=0.7)
        ks_p, t_p=stat_tests(prod_l,non_prod_l)
        ax.annotate('KS_p_value=%s\nt-test_p_value=%s' %(ks_p, t_p), xy=(0.95, 0.95), xycoords='axes fraction', fontsize=8,
        horizontalalignment='right', verticalalignment='top', fontweight='bold')
        
    else:
        plot=ax.hist(prod_l, color='blue', bins=50)
    ax.set_title(str(function_name), fontsize=8)
    ax.set_ylabel('Frequency', fontsize=7)
    ax.tick_params(labelsize=6)
    ax.set_yscale(yscale)
    if p==0:
        ax.legend(loc='upper center', fontsize=6)
    return plot

In [ ]:
def stat_tests(prod_l,non_prod_l):
    ks_s,ks_p=stats.ks_2samp(prod_l,non_prod_l)
    t_s,t_p=stats.ttest_ind(prod_l,non_prod_l)
    if ks_p<=10**-4:
        ks_p='<10^-4'
    else:
        ks_p=str(round(ks_p,4))
    if t_p<=10**-4:
        t_p='<10^-4'
    else:
        t_p=str(round(t_p,4))
    return ks_p, t_p
    

In [ ]:
plot_res_df(general_res_df, general_function_list, 'General')

In [ ]:
plt.close()

In [ ]:
perProd_res_df.head()

In [ ]:

plot_1_res_df(perProd_res_df, percProd_func_list, 'Percent Productive')

In [ ]:
plot_res_df(clonality_res_df, clonality_func_list, 'Clonality')

In [ ]:
plot_res_df(clonality_res_df, clonality_func_list, 'Clonality', 'log', 0.68)

In [ ]:
general_res_df.head()

In [ ]:
corr_df=general_res_df.corr()

In [ ]:
print corr_df

In [ ]:
plot_res_df(diversity_res_df, diversity_func_list, 'Diversity-linear', 'linear', 0.36)

In [ ]:
plot_res_df(diversity_res_df, diversity_func_list, 'Diversity-log', 'log', 0.36)

In [ ]:
plot_res_df(length_res_df, length_func_list, 'length-linear', 'linear', 0.66)

In [ ]:
plot_res_df(length_res_df, length_func_list, 'length-log', 'log', 0.66)

In [ ]:
import time
print "Current date "  + time.strftime("%x")

In [ ]:
cdate=str(time.strftime("%x"))

In [ ]:
print cdate

In [ ]:
length_res_fig_linear=plot_res_df(length_res_df, length_func_list, 'length-linear', 'linear', 0.66)

In [ ]:
length_res_fig_log=plot_res_df(length_res_df, length_func_list, 'length-log', 'log', 0.66)

In [ ]:
diversity_res_fig_linear=plot_res_df(diversity_res_df, diversity_func_list, 'Diversity-linear', 'linear', 0.36)

In [ ]:
diversity_res_fig_log=plot_res_df(diversity_res_df, diversity_func_list, 'Diversity-log', 'log', 0.36)

#### plotting public analysis results

public_res_df_combined is loaded from pickle, the public functions and function list is copied from the 'public' notebook. 

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/public_analysis/calculate_public_stats/public_res_df_combined', 'rb') as f:
    public_res_df_combined=pickle.load(f)
f.close()

In [ ]:
public_res_df_combined.head()

In [ ]:
## public population functions:


def perc_public(df):
    n_public=len(df[df['n_samples']>1])
    perc_public=(float(n_public)/len(df['n_samples']))*100
    return perc_public

def public10perc(df):
    n_public10=len(df[df['n_samples']>57])
    perc_public10=(float(n_public10)/len(df['n_samples']))*100
    return perc_public10

def public50perc(df):
    n_public50=len(df[df['n_samples']>286])
    perc_public50=(float(n_public50)/len(df['n_samples']))*100
    return perc_public50

def public95perc(df):
    n_public95=len(df[df['n_samples']>544])
    perc_public95=(float(n_public95)/len(df['n_samples']))*100
    return perc_public95

def meanSharedSamples(df):
    mean_shared=df['n_samples'].mean()
    return mean_shared

def cdr3PriToPub(df):
    mean_cdr3_pri=df[df['n_samples']==1]['cdr3Length'].mean()
    mean_cdr3_pub=df[df['n_samples']>1]['cdr3Length'].mean()
    cdr3PriToPub=float(mean_cdr3_pri)/mean_cdr3_pub
    return cdr3PriToPub

def cdr3PriToPub95(df):
    mean_cdr3_pri=df[df['n_samples']==1]['cdr3Length'].mean()
    mean_cdr3_pub95=df[df['n_samples']>544]['cdr3Length'].mean()
    cdr3PriToPub95=float(mean_cdr3_pri)/mean_cdr3_pub95
    return cdr3PriToPub95

In [ ]:
public_res_df_combined_toPlot=public_res_df_combined.drop(['public50perc_df_0','public50perc_df_1','public95perc_df_0','public95perc_df_1','cdr3PriToPub95_df_0','cdr3PriToPub95_df_1'], axis=1)
## remove features which have data only for productive sequences (those features are plotted seperately)

In [ ]:
columns_to_keep=['public50perc_df_0','public95perc_df_0', 'cdr3PriToPub95_df_0']
public_res_df_combined_toPlot1=public_res_df_combined[columns_to_keep]
## generate df for features that have data only in productive sequences

In [ ]:
public_res_df_combined_toPlot.head()

In [ ]:
public_func_list_toPlot=[perc_public, public10perc, meanSharedSamples, cdr3PriToPub]

In [ ]:
public_func_list_toPlot1=[public50perc, public95perc, cdr3PriToPub95]

In [ ]:
public_plot=plot_res_df(public_res_df_combined_toPlot, public_func_list_toPlot, 'Public-linear', 'linear', 0.20)

In [ ]:
public_plot1=plot_1_res_df(public_res_df_combined_toPlot1, public_func_list_toPlot1, 'Public-linear', 'linear')

In [ ]:
plot_res_df(public_res_df_combined_toPlot, public_func_list_toPlot, 'Public-log', 'log', 0.20)

#### Generating A figures PDF:

In [ ]:
cdate=str(time.strftime("%d%m%Y"))
create_pdf=True
figlist=[perc_prod_fig, general_res_fig_linear, general_res_fig_log, clonality_res_fig_linear, clonality_res_fig_log, diversity_res_fig_linear,
        diversity_res_fig_log,length_res_fig_linear, length_res_fig_log, public_plot, public_plot1]

if create_pdf:
    with PdfPages('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Images/population_View_%s_samples_%s.pdf' %(n_samples, cdate)) as pdf:
        for fig in figlist:
            pdf.savefig(fig)
    pdf.close
else:
    plt.show()

In [ ]:
plt.savefig('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Images/Public-linear population features-1', dpi=300)

'''
    name, dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)
'''

In [ ]:
plt.savefig('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Images/Public-linear population features', dpi=600)

#### combining all results dataframes:

In [ ]:
public_result_with_tags_df=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/public_result_with_tags_df', "rb"))
public_result_with_tags_df.head()

In [ ]:
length_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/length_res_df_573_samples', "rb"))
clonality_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/clonality_res_df_573_samples', "rb"))
perProd_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/perProd_res_df_573_samples', "rb"))
general_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/general_res_df_573_samples', "rb"))
diversity_res_df_573_samples=pickle.load( open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/diversity_res_df_573_samples', "rb"))






In [ ]:
combined_res_with_tags_df=pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(general_res_df_573_samples,perProd_res_df_573_samples,how='outer', 
                                                     right_index=True,left_index=True), clonality_res_df_573_samples,how='outer',
                                                      right_index=True,left_index=True),length_res_df_573_samples, how='outer',
                                                      right_index=True,left_index=True),diversity_res_df_573_samples, how='outer',
                                                      right_index=True,left_index=True),public_result_with_tags_df,how='outer',
                                                      right_index=True,left_index=True)

In [ ]:
combined_res_with_tags_describe=combined_res_with_tags_df.describe()

In [ ]:
excel_writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/combined_res_with_tags_describe.xlsx'
combined_res_with_tags_describe.to_excel(excel_writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

In [ ]:
import time
cdate=str(time.strftime("%d%m%Y"))
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/combined_res_with_tags_df_%s' %cdate, 'wb') as f:
        pickle.dump(combined_res_with_tags_df,f)
f.close()

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/combined_res_with_tags_df_08032017', 'rb') as f:
        combined_res_with_tags_df=pickle.load(f)
f.close()

In [ ]:
combined_res_with_tags_df.columns.values

#### calculating correlations over the result df:
the significance is tested by both p-value and by permutation analysis:

##### define parameters: 

In [ ]:
df=combined_res_with_tags_df #define the results df
corr_col='perc_public_df_0' #define the feature to calculate correlations with
n_per=10 #define the number of permutations

##### calculate real correlations and p-values

In [ ]:
def calc_sig_corr(df, corr_col, n_per):
    from scipy.stats import pearsonr

    ## stage 1: calculate real correlations and generate dataframe
    print 'calculating real correlations'
    corr_list=[]
    n_tests=len(df.columns.values)
    for column in list(df.columns.values):
        if df[column].dtype == np.float64 or df[column].dtype == np.int64 :
            n1=np.isnan(df[column])
            n2=np.isnan(df[corr_col])
            n=n1|n2
            newx=list(df[column][~n])
            newy=list(df[corr_col][~n])
            r,p = pearsonr(newx,newy)
        else:
            r=np.nan
            p=np.nan
        corr_list.append({'column':column,'real_r':r, 'real_p':p})
    res_corr_df=pd.DataFrame(corr_list)
    res_corr_df.sort(columns='real_p', inplace=1)


    ## stage 2: permutate result df and calculate correlations over permutated data:
    for i in range(n_per):
        print i
        print 'start shuffling df'
        shuffle=df.apply(np.random.permutation)
        shuf_corr_list=[]
        #column_list=[]
        for column in list(shuffle.columns.values):
            #print ('start calculating r for column %s' %column)
            #column_list.append(column)
            if shuffle[column].dtype == np.float64 or shuffle[column].dtype == np.int64 :
                n1=np.isnan(shuffle[column])
                n2=np.isnan(shuffle[corr_col])
                n=n1|n2
                newx=list(shuffle[column][~n])
                newy=list(shuffle[corr_col][~n])
                r,p = pearsonr(newx,newy)
                shuf_corr_list.append(r)
            else:
                shuf_corr_list.append(np.nan)
        #shuffle_r_df.loc[:,('column_shuf_%s'%i)] = column_list
        res_corr_df.loc[:,('r_shuf_%s'%i)] = shuf_corr_list
    
    ## stage 3: calculate confidence interval for shuffled r's:
    #print shuffle_r_df.columns.values
    col_for_percentile=[col for col in res_corr_df.columns.values if col.startswith('r_shuf_')]
    #shuffle_r_df.loc[:,'r_mean']=np.mean(shuffle_r_df[col_for_percentile])
    #res_corr_df['avg'] = res_corr_df[col_for_percentile].mean(axis=1)
    res_corr_df['r_perc_2_5'] = res_corr_df[col_for_percentile].quantile(q=0.025,axis=1)
    res_corr_df['r_perc_97_5'] = res_corr_df[col_for_percentile].quantile(q=0.975,axis=1)
    
    ## stage 4: check significance of the correlations:
    res_correct_p=0.05/n_tests
    res_corr_df['real_p_sig']=np.where(res_corr_df['real_p']<0.05,1,0)
    res_corr_df['real_p_sig_corrected']=np.where(res_corr_df['real_p']<res_correct_p,1,0)
    res_corr_df['r_outof_CI']=np.where((res_corr_df['real_r']<res_corr_df['r_perc_2_5']) | (res_corr_df['real_r']>res_corr_df['r_perc_97_5']) ,1,0)
    col_to_keep=['column','real_p','real_r','r_perc_2_5','r_perc_97_5','real_p_sig','real_p_sig_corrected','r_outof_CI']
    res_corr_df=res_corr_df[col_to_keep]
    res_corr_df.set_index('column',inplace=1)
    only_sig_corr_df=res_corr_df[(res_corr_df['real_p_sig']==1) & (res_corr_df['r_outof_CI']==1)]
    
    return res_corr_df,only_sig_corr_df


In [ ]:
 df10 = pd.DataFrame(np.random.randint(0,100,(100, 100)))

In [ ]:
df10['r_perc_2_5'] = df10.quantile(q=0.025,axis=1)
df10['r_perc_97_5'] = df10.quantile(q=0.975,axis=1)
df10['r_perc_3'] = df10.quantile(q=0.03,axis=1)
df10['r_perc_97'] = df10.quantile(q=0.97,axis=1)


In [ ]:
df10[['r_perc_2_5','r_perc_97_5','r_perc_3','r_perc_97']].mean()

In [ ]:
percPublicProd_corr_df,percPublicProd_SigCorr_df=calc_sig_corr(combined_res_with_tags_df, 'perc_public_df_0', 1000)

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/percPublicProd_corr_df', 'wb') as f:
        pickle.dump(percPublicProd_corr_df,f)
f.close()

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/percPublicProd_corr_df', 'rb') as f:
        percPublicProd_corr_df=pickle.load(f)
f.close()

In [ ]:
percPublicProd_corr_df[50:]

In [ ]:
print len(percPublicProd_SigCorr_df)
print len(percPublicProd_corr_df)

In [ ]:
meanSharedProd_corr_df,meanSharedProd_SigCorr_df=calc_sig_corr(combined_res_with_tags_df, 'meanSharedSamples_df_0', 1000)

In [ ]:
meanSharedProd_SigCorr_df.head()

In [ ]:
print len(meanSharedProd_SigCorr_df)
print len(meanSharedProd_corr_df)

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/meanSharedProd_corr_df', 'wb') as f:
        pickle.dump(meanSharedProd_corr_df,f)
f.close()

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/meanSharedProd_corr_df', 'rb') as f:
        meanSharedProd_corr_df=pickle.load(f)
f.close()

In [ ]:
meanSharedProd_corr_df[:50]

In [ ]:
writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/public_analysis/meanSharedProd_corr_df.xlsx'
meanSharedProd_corr_df.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

In [ ]:
writer='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/public_analysis/percPublicProd_corr_df.xlsx'
percPublicProd_corr_df.to_excel(writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True)

In [ ]:
combined_res_with_tags_df_corr.sort(columns='meanSharedSamples_df_0',ascending=False)['meanSharedSamples_df_0'][:50]
                                                                

In [ ]:
combined_res_with_tags_df_corr.sort(columns='perc_public_df_0',ascending=False)['perc_public_df_0'][0:50]

In [ ]:
combined_res_with_tags_df_corr.sort(columns='perc_public_df_0',ascending=False)['perc_public_df_0'][50:]

In [ ]:
def draw_correlation_scatter(x, y, figsize = (3, 3), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = None, ms=4, logd = False,\
                             xlab = None, ylab = None, filename = None, title = None,
                             color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                             contour = False, **figkwargs):
    from scipy.stats import pearsonr
    fig = plt.figure(figsize = figsize, dpi = dpi)
    axB = fig.add_subplot(111)
    if contour:
        print "Contour plot are experimental here"
        axB.hist2d(x,y,bins = 40,norm=LogNorm())
    else:
        axB.plot(x, y, 'o', color = color, ms=ms, **figkwargs)
    if logd:
        axB.set_xscale('log',basex=2)
        axB.set_yscale('log',basey=2)
    if xticks is not None:
        axB.set_xticks(xticks)
        axB.set_xticklabels(xticks)
    if xticklabels is not None:
        axB.set_xticklabels(xticklabels)
    if yticks is not None:
        axB.set_yticks(yticks)
        axB.set_yticklabels(yticks)
    if xlim is not None:
        axB.set_xlim(xlim)
    if ylim is not None:
        axB.set_ylim(ylim)
    if r is not None: 
        if r == True:
            n=np.isnan(x)
            newx=list(x[~n])
            newy=list(y[~n])
            r,p = pearsonr(newx,newy)
        axB.text(0.01,0.99,"r=%.4f p=%.4f" %(r,p), transform=axB.transAxes, verticalalignment = 'top', ha = 'left',fontsize=14,color='red')
    if xlab is not None:
        axB.set_xlabel(xlab)
    if ylab is not None:
        axB.set_ylabel(ylab)
    if title is not None:
        plt.title(title, fontsize=16)
    if grid:
        axB.grid()
    if filename is not None:
        fig.savefig(filename, bbox_inches='tight', dpi = dpi)
    return fig, axB


In [ ]:
def draw_multiple_correlation_scatter(x, y, ynames=None, figsize = (4, 4), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = None, ms=4, logd = False,\
                             xlab = None, ylab = None, filename = None, title = None,
                             color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                             contour = False, **figkwargs):
    from scipy.stats import pearsonr
    fig = plt.figure(figsize = figsize, dpi = dpi)
    axB = fig.add_subplot(111)
    if contour:
        print "Contour plot are experimental here"
        axB.hist2d(x,y,bins = 40,norm=LogNorm())
    else:
        for i in range(len(y)):
            axB.plot(x, y[i], 'o', color = color[i], ms=ms[i], label=ynames[i], **figkwargs)
    axB.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    #axB.legend(loc='best',fontsize=10)
    if logd:
        axB.set_xscale('log',basex=2)
        axB.set_yscale('log',basey=2)
    if xticks is not None:
        axB.set_xticks(xticks)
        axB.set_xticklabels(xticks)
    if xticklabels is not None:
        axB.set_xticklabels(xticklabels)
    if yticks is not None:
        axB.set_yticks(yticks)
        axB.set_yticklabels(yticks)
    if xlim is not None:
        axB.set_xlim(xlim)
    if ylim is not None:
        axB.set_ylim(ylim)
    if r is not None: 
        if r == True:
            n=np.isnan(x)
            newx=list(x[~n])
            newy_l=[]
            r_l=[]
            p_l=[]
            posR=0.99
            for i in range(len(y)):
                newy=list(y[i][~n])
                r,p = pearsonr(newx,newy)
                newy_l.append(newy)
                r_l.append(r)
                p_l.append(p)
                axB.text(0.01,posR,"r=%.4f p=%.4f" %(r,p), transform=axB.transAxes, verticalalignment = 'top', ha = 'left',fontsize=10,color=color[i])
                posR=posR-0.03
    if xlab is not None:
        axB.set_xlabel(xlab)
    if ylab is not None:
        axB.set_ylabel(ylab)
    if title is not None:
        plt.title(title, fontsize=16)
    if grid:
        axB.grid()
    if filename is not None:
        fig.savefig(filename, bbox_inches='tight', dpi = dpi)
    return fig, axB


In [ ]:
combined_res_with_tags_noOutliers=combined_res_with_tags_df[combined_res_with_tags_df['perc_public_df_0']<80]

In [ ]:
Age_nShared_correl_fig=draw_correlation_scatter(combined_res_with_tags_df['Age'], combined_res_with_tags_df['perc_public_df_0'], figsize = (6, 6), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=6, logd = False,\
                             xlab = 'Age', ylab = 'Percent Public-Productive sequences', filename = 'Age-Percent Public correlation', title = 'Age-Percent Public correlation',
                             color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
Age_nShared_correl_no_outliers_fig=draw_correlation_scatter(combined_res_with_tags_noOutliers['Age'], combined_res_with_tags_noOutliers['perc_public_df_0'], figsize = (6, 6), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=6, logd = False,\
                             xlab = 'Age', ylab = 'Percent Public-Productive sequences', filename = 'Age-Percent Public correlation-noOutliers', title = 'Age-Percent Public correlation (no outliers)',
                             color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
Age_meanShared_correl_fig=draw_correlation_scatter(combined_res_with_tags_df['Age'], combined_res_with_tags_df['meanSharedSamples_df_0'], figsize = (6, 6), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=6, logd = False,\
                             xlab = 'Age', ylab = 'Mean shared samples-Productive sequences', filename = 'Age-Mean shared samples correlation', title = 'Age-Mean shared samples correlation',
                             color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
Age_meanShared_correl_no_outliers_fig=draw_correlation_scatter(combined_res_with_tags_noOutliers['Age'], combined_res_with_tags_noOutliers['meanSharedSamples_df_0'], figsize = (6, 6), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=6, logd = False,\
                             xlab = 'Age', ylab = 'Mean shared samples-Productive sequences', filename = 'Age-Mean shared samples correlation-noOutliers', title = 'Age-Mean shared samples correlation correlation (no outliers)',
                             color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
meanclonalaa_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['mean_clonal_aa_df_0'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'mean clonality (%, productive aa sequences)', ylab = '', filename = 'mean_clonal aa-sharing correlation', title = 'mean_clonal aa-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
topclonalaa_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['top_clonal_aa_df_0'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'highest clonality (%, productive aa sequences)', ylab = '', filename = 'top_clonal aa-sharing correlation', title = 'top_clonal aa-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
top_clonal_aa_df_0

In [ ]:
meanclonalnt_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['mean_clonal_nt_df_0'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'mean clonality (%, productive nt sequences)', ylab = '', filename = 'mean_clonal_nt-sharing correlation', title = 'mean_clonal nt-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
topclonalnt_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['top_clonal_nt_df_0'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'highest clonality (%, productive nt sequences)', ylab = '', filename = 'top_clonal_nt-sharing correlation', title = 'top_clonal nt-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
top1000clonsNonProd_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['top_1000clons_aa_df_1'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'fraction of top 1000 clones (%, non-productive nt sequences)', ylab = '', filename = 'top1000clonsNonProd-sharing correlation', title = 'top 1000 clons Non Prod-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()


In [ ]:
cdr3priToPub_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['cdr3PriToPub_df_0'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'ratio', ylab = '', filename = 'cdr3PriToPub_prod-sharing correlation', title = 'cdr3PriToPub_prod-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
meanCDR3_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['mean_cdr3_df_0'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'Cdr3 length (bps)', ylab = '', filename = 'mean_cdr3_prod-sharing correlation', title = 'mean_cdr3_prod-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()

In [ ]:
normUniqueAA_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['norm_uniqe_aa_sequences_df_0'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'number of unique aa sequences per 5,000 sequences ', ylab = '', filename = 'norm_unique_aa_prod-sharing correlation', title = 'norm_unique_aa_prod-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()


In [ ]:
normUniqueNT_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['norm_uniqe_nt_sequences_df_0'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'number of unique nt sequences per 100,000 sequences ', ylab = '', filename = 'norm_unique_nt_prod-sharing correlation', title = 'norm_unique_nt_prod-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()


In [ ]:
meanN1Insertion_2sharing_correl_fig=draw_multiple_correlation_scatter(combined_res_with_tags_df['mean_n1Insertion_df_0'], [combined_res_with_tags_df['meanSharedSamples_df_0'], combined_res_with_tags_df['perc_public_df_0']], ynames=['meanSharedSamplesProd','perc_public_Prod'], figsize = (8, 8), xticks=None, yticks=None,\
                             xlim = None, ylim = None, r = True, ms=[6,6], logd = False,\
                             xlab = 'insertion length (bps)', ylab = '', filename = 'mean_n1Insertion_prod-sharing correlation', title = 'mean_n1Insertion_prod-sharing correlation',
                             color = ['black','green'], grid = True, dpi = 800, xticklabels = None, 
                             contour = False)
plt.show()


#### calculating gender differences for all features:

In [ ]:
from scipy.stats import ttest_ind

males = combined_res_with_tags_df[combined_res_with_tags_df['Gender']=='Male']
females = combined_res_with_tags_df[combined_res_with_tags_df['Gender']=='Female']


ttest_list=[]
for column in list(combined_res_with_tags_df.columns.values):
    if column == 'Gender' or column == '_merge' :
        print column
    else:
        t,p=ttest_ind(males[column], females[column])
        ttest_list.append({'column':column,'p-value':p})
genderDiff_df=pd.DataFrame(ttest_list)
genderDiff_df.sort(columns='p-value', inplace=1)






In [ ]:
genderDiff_df.head()

In [ ]:
genderDiff_df['sig']=np.where(genderDiff_df['p-value']<0.0005, 1, 0)

In [ ]:
from statsmodels.sandbox.stats.multicomp import multipletests
pvals=list(genderDiff_df['p-value'])
reject,BHpval_df,x,y=multipletests(pvals, alpha=0.01, method='hs', returnsorted=False)
reject,bonferroni_df,x,y=multipletests(pvals, alpha=0.01, method='bonferroni', returnsorted=False)
genderDiff_df['BH FDR']=BHpval_df
genderDiff_df['bonferroni FDR']=bonferroni_df

In [ ]:
genderDiff_df[50:]

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/genderDiff_df', 'wb') as f:
        pickle.dump(genderDiff_df, f)
f.close()

In [ ]:
gcMaleMean=np.mean(males['gc_content_df_0'])
gcFemaleMean=np.mean(females['gc_content_df_0'])
gcMaleSTD=np.std(males['gc_content_df_0'])
gcFemaleSTD=np.std(females['gc_content_df_0'])
means=[gcMaleMean,gcFemaleMean]
stds=[gcMaleSTD,gcFemaleSTD ]


plt.errorbar([1,2], means, stds, linestyle='None', marker='^')
plt.xticks([1,2],['Males','Females'])
plt.xlim(0.5,2.5)
plt.title('Gender effect on GC% in productive sequences')
plt.xlabel('Gender')
plt.ylabel('GC%')

plt.ylim(55,59)
plt.show()

In [ ]:
top1000MaleMean=np.mean(males['top_1000clons_nt_df_1'])
top1000FemaleMean=np.mean(females['top_1000clons_nt_df_1'])
top1000MaleSTD=np.std(males['top_1000clons_nt_df_1'])
top1000FemaleSTD=np.std(females['top_1000clons_nt_df_1'])
means=[top1000MaleMean,top1000FemaleMean]
stds=[top1000MaleSTD,top1000FemaleSTD ]


plt.errorbar([1,2], means, stds, linestyle='None', marker='^')
plt.xticks([1,2],['Males','Females'])
plt.xlim(0.5,2.5)
plt.title('Gender effect on the top 1000 clone fraction in non-productive sequences')
plt.xlabel('Gender')
plt.ylabel('fraction')
plt.show()

In [ ]:
uniqueAAMaleMean=np.mean(males['unique_aa_n_df_0'])
uniqueAAFemaleMean=np.mean(females['unique_aa_n_df_0'])
uniqueAAMaleSTD=np.std(males['unique_aa_n_df_0'])
uniqueAAFemaleSTD=np.std(females['unique_aa_n_df_0'])
means=[uniqueAAMaleMean,uniqueAAFemaleMean]
stds=[uniqueAAMaleSTD,uniqueAAFemaleSTD ]


plt.errorbar([1,2], means, stds, linestyle='None', marker='^')
plt.xticks([1,2],['Males','Females'])
plt.xlim(0.5,2.5)
plt.title('Gender effect on the number of unique aa sequences')
plt.xlabel('Gender')
plt.ylabel('sequence number')
plt.show()

In [ ]:
percPublicMaleMean=np.mean(males['perc_public_df_0'])
percPublicFemaleMean=np.mean(females['perc_public_df_0'])
percPublicMaleSTD=np.std(males['perc_public_df_0'])
percPublicFemaleSTD=np.std(females['perc_public_df_0'])
means=[percPublicMaleMean,percPublicFemaleMean]
stds=[percPublicMaleSTD,percPublicFemaleSTD ]

GenderOnPercPublicProdFig = plt.figure(figsize = (4,4))
plt.errorbar([1,2], means, stds, linestyle='None', marker='^',capsize=10, capthick=2, markerfacecolor='black',elinewidth=2, markersize=9, color='red')
plt.xticks([1,2],['Males','Females'])
plt.xlim(0.5,2.5)
plt.title('Gender effect on the public sequence fraction in productive sequences')
plt.xlabel('Gender')
plt.ylabel('fraction')
GenderOnPercPublicProdFig.savefig('GenderOnPercPublicProdFig', bbox_inches='tight', dpi = 800)
plt.show()

In [ ]:
percPublicMaleMean=np.mean(males['meanSharedSamples_df_0'])
percPublicFemaleMean=np.mean(females['meanSharedSamples_df_0'])
percPublicMaleSTD=np.std(males['meanSharedSamples_df_0'])
percPublicFemaleSTD=np.std(females['meanSharedSamples_df_0'])
means=[percPublicMaleMean,percPublicFemaleMean]
stds=[percPublicMaleSTD,percPublicFemaleSTD ]


GenderOnMeanSharedSamplesProdFig = plt.figure(figsize = (4,4))
plt.errorbar([1,2], means, stds, linestyle='None', marker='^',capsize=10, capthick=2, markerfacecolor='black',elinewidth=2, markersize=9, color='red')
plt.xticks([1,2],['Males','Females'])
plt.xlim(0.5,2.5)
plt.title('Gender effect on the the mean number of shared samples per sequence')
plt.xlabel('Gender')
plt.ylabel('mean shared samples number')
GenderOnMeanSharedSamplesProdFig.savefig('GenderOnMeanSharedSamplesProdFig', bbox_inches='tight', dpi = 800)
plt.show()

#### calculating correlations between all parameters in the result df:

In [ ]:
l=['a','b','c','d']
for n1,i1 in enumerate(l):
    for n2,i2 in enumerate(l[n1+1:]):
        print n1,i1,n2,i2


In [ ]:
def calc_sig_corr_all2all(df, n_per):
    from scipy.stats import pearsonr

    ## stage 1: calculate real correlations and generate dataframe
    print 'calculating real correlations...'
    corr_list=[]
    for n1, column1 in enumerate(list(df.columns.values)):  
        for n2, column2 in enumerate(list(df.columns.values)[n1+2:]):
            if (df[column1].dtype == np.float64 or df[column1].dtype == np.int64)&(df[column2].dtype == np.float64 or df[column2].dtype == np.int64) :
                nc1=np.isnan(df[column1])
                nc2=np.isnan(df[column2])
                n=nc1|nc2
                newx=list(df[column1][~n])
                newy=list(df[column2][~n])
                r,p = pearsonr(newx,newy)
            else:
                r=np.nan
                p=np.nan
            corr_list.append({'column1':column1,'column2':column2, 'real_r':r, 'real_p':p})
    print 'finished calculating real correlations'
    res_corr_df=pd.DataFrame(corr_list)
    res_corr_df['abs_r'] = res_corr_df['real_r'].abs()
    res_corr_df.sort_index(by=['real_p','abs_r','column1','column2'], ascending=[True,False,True,True],inplace=1)
    res_corr_df.drop('abs_r', axis=1,inplace=1)


## stage 2: permutate result df and calculate correlations over permutated data:
    print 'calculating suffle correlations...'
    for i in range(n_per):
        print i
        print 'start shuffling df'
        shuffle=df.apply(np.random.permutation)
        shuf_corr_list=[]
        #column_list=[]
        for n1, column1 in enumerate(list(shuffle.columns.values)):
            for n2, column2 in enumerate(list(shuffle.columns.values)[n1+2:]):
                if (shuffle[column1].dtype == np.float64 or shuffle[column1].dtype == np.int64)&(shuffle[column2].dtype == np.float64 or shuffle[column2].dtype == np.int64):
                    nc1=np.isnan(shuffle[column1])
                    nc2=np.isnan(shuffle[column2])
                    n=nc1|nc2
                    newx=list(shuffle[column1][~n])
                    newy=list(shuffle[column2][~n])
                    r,p = pearsonr(newx,newy)
                    shuf_corr_list.append(r)
                else:
                    shuf_corr_list.append(np.nan)
            
        res_corr_df.loc[:,('r_shuf_%s'%i)] = shuf_corr_list
    
    print 'finished calculating suffle correlations'
    
    ## stage 3: calculate confidence interval for shuffled r's:
    print 'calculating percentile values...'
    #print shuffle_r_df.columns.values
    col_for_percentile=[col for col in res_corr_df.columns.values if col.startswith('r_shuf_')]
    #shuffle_r_df.loc[:,'r_mean']=np.mean(shuffle_r_df[col_for_percentile])
    #res_corr_df['avg'] = res_corr_df[col_for_percentile].mean(axis=1)
    res_corr_df['r_perc_2_5'] = res_corr_df[col_for_percentile].quantile(q=0.025,axis=1)
    res_corr_df['r_perc_97_5'] = res_corr_df[col_for_percentile].quantile(q=0.975,axis=1)
    print 'finished calculating percentile values'
    
    ## stage 4: check significance of the correlations:
    n_tests=sigma(len(df.columns.values))
    res_correct_p=0.05/n_tests
    res_corr_df['real_p_sig']=np.where(res_corr_df['real_p']<0.05,1,0)
    res_corr_df['real_p_sig_corrected']=np.where(res_corr_df['real_p']<res_correct_p,1,0)
    res_corr_df['r_outof_CI']=np.where((res_corr_df['real_r']<res_corr_df['r_perc_2_5']) | (res_corr_df['real_r']>res_corr_df['r_perc_97_5']) ,1,0)
    col_to_keep=['column1','column2','real_p','real_r','r_perc_2_5','r_perc_97_5','real_p_sig','real_p_sig_corrected','r_outof_CI']
    res_corr_df=res_corr_df[col_to_keep]
    res_corr_df.set_index('column1',inplace=1)
    res_corr_df=res_corr_df[~np.isnan(res_corr_df['real_r'])]
    only_sig_corr_df=res_corr_df[(res_corr_df['real_p_sig']==1) & (res_corr_df['r_outof_CI']==1)]
    
    return res_corr_df,only_sig_corr_df


def sigma(x):
    sigma=0
    for n in range(x):
        sigma=+n
    return sigma


In [ ]:
res_corr_df,only_sig_corr_df =calc_sig_corr_all2all(combined_res_with_tags_df, 100)

In [ ]:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_demo_data/Pickles/res_corr_df', 'wb') as f:
        pickle.dump(res_corr_df, f)
f.close()

In [ ]:
print len(res_corr_df)
print len(only_sig_corr_df)

In [ ]:
only_sig_corr_df[['column2','real_r','real_p','real_p_sig','real_p_sig_corrected','r_outof_CI']][:20]
                                                                                                 

In [ ]:
meanClonalAA_public95percCorrFig=draw_correlation_scatter(combined_res_with_tags_df['mean_clonal_aa_df_0'], 
                                                combined_res_with_tags_df['public95perc_df_0'], figsize = (6, 6), 
                                                xticks=None, yticks=None,\
                                                 xlim = None, ylim = None, r = True, ms=6, logd = False,\
                                                 xlab = 'Frequency', ylab = 'percent shared among >95% of samples', 
                                                filename = 'mean_clonal_aa_df_0-public95perc_df_0 correlation', title = 'mean_clonal_aa-public95perc correlation (productive)',
                                                 color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                                                 contour = False)
plt.show()

In [ ]:
meanClonalNT_public95percCorrFig=draw_correlation_scatter(combined_res_with_tags_df['mean_clonal_nt_df_0'], 
                                                combined_res_with_tags_df['public95perc_df_0'], figsize = (6, 6), 
                                                xticks=None, yticks=None,\
                                                 xlim = None, ylim = None, r = True, ms=6, logd = False,\
                                                 xlab = 'Frequency', ylab = 'percent shared among >95% of samples', 
                                                filename = 'mean_clonal_nt_df_0-public95perc_df_0 correlation', title = 'mean_clonal_nt-public95perc correlation (productive)',
                                                 color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                                                 contour = False)
plt.show()

In [ ]:
percentPublic_CDR3ratio_prod_corrFig=draw_correlation_scatter(combined_res_with_tags_df['perc_public_df_0'], 
                                                combined_res_with_tags_df['cdr3PriToPub_df_0'], figsize = (6, 6), 
                                                xticks=None, yticks=None,\
                                                 xlim = None, ylim = None, r = True, ms=6, logd = False,\
                                                 xlab = 'percent public', ylab = 'CDR3 ratio', 
                                                filename = 'percent public-CDR3 ratio_prod_corr', title = 'percent public-CDR3 ratio correlation (productive)',
                                                 color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                                                 contour = False)
plt.show()

In [ ]:
mean_n1_n2_insertion_corr_prod_Fig=draw_correlation_scatter(combined_res_with_tags_df['mean_n1Insertion_df_0'], 
                                                combined_res_with_tags_df['mean_n2Insertion_df_0'], figsize = (6, 6), 
                                                xticks=None, yticks=None,\
                                                 xlim = None, ylim = None, r = True, ms=6, logd = False,\
                                                 xlab = 'mean n1Insertion (bps)', ylab = 'mean n2Insertion (bps)', 
                                                filename = 'mean_n1_n2_insertion_corr_prod', title = 'mean n1-n2 insertion correlation (productive)',
                                                 color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                                                 contour = False)
plt.show()

In [ ]:
mean_Jdel_N2ins_corr_prod_Fig=draw_correlation_scatter(combined_res_with_tags_df['mean_jDeletion_df_0'], 
                                                combined_res_with_tags_df['mean_n2Insertion_df_0'], figsize = (6, 6), 
                                                xticks=None, yticks=None,\
                                                 xlim = None, ylim = None, r = True, ms=6, logd = False,\
                                                 xlab = 'mean jDeletion (bps)', ylab = 'mean n2Insertion (bps)', 
                                                filename = 'mean_jDEL_n2_insertion_corr_prod', title = 'mean j deletion-n2 insertion correlation (productive)',
                                                 color = "#a0a0a0", grid = True, dpi = 800, xticklabels = None, 
                                                 contour = False)
plt.show()

In [ ]:
res_corr_df[res_corr_df.index=='mean_cdr3_df_0']

In [ ]:
print 'dsgfd'